In [ ]:
import os
import sys
cwd = os.getcwd()
if os.path.basename(cwd) == 'models':
    models_dir   = cwd
    project_root = os.path.dirname(cwd)
else:
    project_root = cwd
    models_dir   = os.path.join(project_root, 'models')

new_path = [models_dir]
for p in sys.path:
    if p in (models_dir, project_root) or p == '':
        continue
    new_path.append(p)
new_path.append(project_root)
sys.path[:] = new_path

# CWD hacky solution
print("Python path:", sys.path)
print("Now CWD is:", os.getcwd())


Python path: ['/Users/zackeryhe/Desktop/sophomore/cs178/178-final/models', '/Users/zackeryhe/Desktop/sophomore/cs178/178-final/models/models', '/opt/homebrew/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/opt/homebrew/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '/Users/zackeryhe/Desktop/sophomore/cs178/178-final/.venv/lib/python3.12/site-packages', '/Users/zackeryhe/Desktop/sophomore/cs178/178-final']
Now CWD is: /Users/zackeryhe/Desktop/sophomore/cs178/178-final/models


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from utils.mnist_reader import load_mnist
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.model_selection import learning_curve
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    precision_recall_curve, PrecisionRecallDisplay
)

# change back
cwd = os.getcwd()
print("Now CWD is:", os.getcwd())

Now CWD is: /Users/zackeryhe/Desktop/sophomore/cs178/178-final


In [13]:
class KNNModel:
    def __init__(self, n_neighbors: int = 5, **kwargs):
        self.model = KNeighborsClassifier(n_neighbors=n_neighbors, **kwargs)

    def fit(self, X, y):
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)

    def score(self, X, y):
        return self.model.score(X, y)

    def plot_learning_curve(self, X, y, cv=5, scoring='accuracy',
                            train_sizes=np.linspace(0.1, 1.0, 5)):
        train_sizes, train_scores, val_scores = learning_curve(
            self.model, X, y,
            cv=cv,
            scoring=scoring,
            train_sizes=train_sizes,
            n_jobs=-1,
            shuffle=True,
            random_state=1234
        )
        train_mean = np.mean(train_scores, axis=1)
        val_mean   = np.mean(val_scores, axis=1)

        plt.figure()
        plt.plot(train_sizes, train_mean, 'o-', label="Train score")
        plt.plot(train_sizes, val_mean,   'o-', label="CV score")
        plt.xlabel("Training examples")
        plt.ylabel(scoring.capitalize())
        plt.title("Learning Curve")
        plt.legend(loc="best")
        plt.grid(True)
        plt.show()

    def plot_confusion_matrix(self, X, y, normalize=None, cmap=plt.cm.Blues):
        y_pred = self.predict(X)
        cm = confusion_matrix(y, y_pred, normalize=normalize)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                      display_labels=self.model.classes_)
        plt.figure()
        disp.plot(cmap=cmap, values_format='.2f' if normalize else 'd',
                  xticks_rotation=45)
        plt.title("Confusion Matrix" + (f" (normalized)" if normalize else ""))
        plt.show()

    def plot_precision_recall_curve(self, X, y, average='micro'):
        if not hasattr(self.model, "predict_proba"):
            raise AttributeError("Model has no `predict_proba`—cannot get scores.")
        
        y_bin = label_binarize(y, classes=self.model.classes_)
        y_score = self.model.predict_proba(X)

        precision, recall, _ = precision_recall_curve(
            y_bin.ravel(), y_score.ravel()
        )
        disp = PrecisionRecallDisplay(precision=precision, recall=recall)
        plt.figure()
        disp.plot()
        plt.title(f"Precision–Recall Curve ({average}-average)")
        plt.show()

    def plot_error_rate_vs_k(self, X_train, y_train, X_test, y_test, k_range=None):
        """
        Since K-NN has no training epochs, plot error rate vs. number of neighbors (k).
        """
        if k_range is None:
            k_range = range(1, 21)
        error_rates = []
        for k in k_range:
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, y_train)
            error = 1 - knn.score(X_test, y_test)
            error_rates.append(error)

        plt.figure()
        plt.plot(list(k_range), error_rates, 'o-')
        plt.xlabel("Number of neighbors (k)")
        plt.ylabel("Error rate")
        plt.title("Error Rate vs. k")
        plt.grid(True)
        plt.show()


In [ ]:
X_train, y_train = load_mnist('data/fashion', kind='train')
X_test,  y_test  = load_mnist('data/fashion', kind='t10k')


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

knn = KNNModel(n_neighbors=5)
knn.fit(X_train, y_train)

FileNotFoundError: [Errno 2] No such file or directory: 'data/fashion/train-labels-idx1-ubyte.gz'

In [ ]:

print("Train accuracy:", knn.score(X_train, y_train))
print("Test accuracy: ", knn.score(X_test,  y_test))


In [ ]:
knn.plot_learning_curve(X_train, y_train)

In [ ]:
knn.plot_confusion_matrix(X_test, y_test, normalize='true')

In [ ]:
knn.plot_precision_recall_curve(X_test, y_test)

In [ ]:
knn.plot_error_rate_vs_k(X_train, y_train, X_test, y_test, k_range=range(1, 21))